<a href="https://colab.research.google.com/github/nmazshellmar/labs-for-Data-processing-structures-and-algorithms/blob/main/C%C3%B3pia_de_%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0__Lab3_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 3.
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы ФИО ГРУППА
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом.

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса.

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



In [4]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
import heapq
from IPython.display import display


In [5]:
# Implementação dos algoritmos
# Algoritmo de Dijkstra
def dijkstra(adj_matrix, start):
    num_nodes = len(adj_matrix)
    distances = {node: float('inf') for node in range(num_nodes)}
    distances[start] = 0
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]:
            continue
        for neighbor in range(num_nodes):
            weight = adj_matrix[current_node][neighbor]
            if weight > 0:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))
    return distances

In [6]:
# Algoritmo de Floyd-Warshall
def floyd_warshall(adj_matrix):
    num_nodes = len(adj_matrix)
    dist = adj_matrix.copy()
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if dist[i][k] and dist[k][j]:
                    new_distance = dist[i][k] + dist[k][j]
                    if dist[i][j] == 0 or new_distance < dist[i][j]:
                        dist[i][j] = new_distance
    return dist

In [7]:
# Algoritmo de Bellman-Ford (corrigido)
def bellman_ford_fixed(adj_matrix, start):
    num_nodes = len(adj_matrix)
    distances = {node: float('inf') for node in range(num_nodes)}
    distances[start] = 0
    for _ in range(num_nodes - 1):
        for u in range(num_nodes):
            for v in range(num_nodes):
                weight = adj_matrix[u][v]
                if weight > 0 and distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight
    return {k: (v if v != float('inf') else 0) for k, v in distances.items()}

In [8]:
# Algoritmo de Johnson (corrigido)
def johnson_fixed(adj_matrix):
    num_nodes = len(adj_matrix)
    new_matrix = np.zeros((num_nodes + 1, num_nodes + 1))
    new_matrix[:num_nodes, :num_nodes] = adj_matrix
    for i in range(num_nodes):
        new_matrix[num_nodes, i] = 0
    h = bellman_ford_fixed(new_matrix, num_nodes)
    new_adj_matrix = adj_matrix.copy()
    for u in range(num_nodes):
        for v in range(num_nodes):
            if new_adj_matrix[u, v] > 0 and u in h and v in h:
                new_adj_matrix[u, v] += h[u] - h[v]
    distances = np.zeros((num_nodes, num_nodes))
    for u in range(num_nodes):
        distances[u] = list(dijkstra(new_adj_matrix, u).values())
    return distances

In [9]:
# Algoritmo de Levit
def levit(adj_matrix, start):
    num_nodes = len(adj_matrix)
    distances = {i: float('inf') for i in range(num_nodes)}
    distances[start] = 0
    main_queue = []
    urgent_queue = []
    main_queue.append(start)
    while main_queue or urgent_queue:
        if urgent_queue:
            u = urgent_queue.pop(0)
        else:
            u = main_queue.pop(0)
        for v in range(num_nodes):
            weight = adj_matrix[u][v]
            if weight > 0:
                if distances[v] == float('inf'):
                    distances[v] = distances[u] + weight
                    main_queue.append(v)
                elif distances[v] > distances[u] + weight:
                    distances[v] = distances[u] + weight
                    urgent_queue.append(v)
    return distances

In [10]:
# Análise de desempenho
def analyze_final_corrected_algorithms(adj_matrix):
    num_nodes = len(adj_matrix)
    algorithms = {
        'Floyd-Warshall': floyd_warshall,
        'Dijkstra': lambda mat: dijkstra(mat, 0),
        'Bellman-Ford': lambda mat: bellman_ford_fixed(mat, 0),
        'Johnson (corrigido)': johnson_fixed,
        'Levit': lambda mat: levit(mat, 0)
    }
    results = []
    for algo_name, algo_func in algorithms.items():
        start_time = time.time()
        algo_func(adj_matrix)
        elapsed_time = time.time() - start_time
        results.append((algo_name, num_nodes, elapsed_time))
    df = pd.DataFrame(results, columns=['Algorithm', 'Nodes', 'Time (s)'])
    return df

In [13]:
import numpy as np

# Definição da matriz de adjacência de teste
test_adj_matrix = np.array([
    [0, 10, 0, 30, 100],
    [0, 0, 50, 0, 10],
    [0, 0, 0, 20, 0],
    [0, 0, 0, 0, 60],
    [0, 0, 0, 0, 0]
])


In [14]:
df_final_corrected_analysis = analyze_final_corrected_algorithms(test_adj_matrix)
display(df_final_corrected_analysis)


,Algorithm,Nodes,Time (s)
0,Floyd-Warshall,5,0.006315
1,Dijkstra,5,0.000214
2,Bellman-Ford,5,0.000209
3,Johnson (corrigido),5,0.000336
4,Levit,5,0.000072


### Выполнение:

In [2]:
!pip install ipywidgets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.9 MB/s eta 0:00:00


### Вывод